In [1]:
%matplotlib inline
import numpy as np
from __future__ import division

In [2]:
##Future plans: n^2 by n^2 board for all positive n
##Future plans: make a smarter solver (each square has 9 options, remove options)
class Board:
    def __init__(self):
        self.board = np.zeros((9,9))
    def view(self):
        return np.copy(self.board)
    def set_board(self,vals):
        if type(vals) == str:
            vals = [i for i in vals]
        vals = np.reshape(vals,(9,9))
        for i in range(9):
            self.set_row(vals[i],i+1)
    def set_row(self,vals,row):
        for j in range(9):
            try:
                self.board[row-1][j] = vals[j]
            except:
                self.board[row-1][j] = 0
    def set_column(self,vals,col):
        for i in range(9):
            try:
                self.board[i][col-1] = vals[i]
            except:
                self.board[i][col-1] = 0
    def set_val(self,val,pos):
        try:
            int(val)
        except:
            val = 0
        if type(pos) == tuple:
            self.board[pos[1]-1][pos[0]-1] = val
        else:
            self.board[(pos-1)//9][(pos-1)%9] = val
    def check_solved(self):
        return self.check_legal() and self.check_filled()
    def check_filled(self):
        return np.count_nonzero(self.board) == 81
    def check_legal(self):
        return self.check_rows() and self.check_columns() and self.check_boxes()
    def check_rows(self,transpose=False):
        b2 = np.copy(self.board)
        if transpose:
            b2 = np.transpose(b2)
        for i in range(9):
            num_nonzero = np.count_nonzero(b2[i])
            num_unique = len(np.unique(b2[i]))
            if num_nonzero == 9 and num_unique != 9:
                return False
            if num_nonzero != 9 and num_unique != num_nonzero + 1:
                return False
        return True
    def check_columns(self):
        return self.check_rows(transpose=True)
    def check_boxes(self):
        for i in range(3):
            for j in range(3):
                r1 = self.board[3*i][3*j:3*j+3]
                r2 = self.board[3*i+1][3*j:3*j+3]
                r3 = self.board[3*i+2][3*j:3*j+3]
                num_nonzero = np.count_nonzero(r1) + np.count_nonzero(r2) + np.count_nonzero(r3)
                num_unique = len(set(r1).union(set(r2).union(set(r3))))
                if num_nonzero == 9 and num_unique != 9:
                    return False
                if num_nonzero != 9 and num_unique != num_nonzero + 1:
                    return False
        return True
    def solve(self):
        b2 = np.copy(self.board)
        if not self._solver():
            self.board = b2
            print("Solve failed! Board is impossible to solve.")
        else:
            print("Solve complete!")
        return self.view()
    def _solver(self):
        if self.check_solved():
            return True
        if not self.check_legal():
            return False
        for pos in range(81):
            if self.board[pos//9][pos%9] == 0:
                break
        vals = [x+1 for x in range(9)]
        for val in vals:
            self.board[pos//9][pos%9] = val
#             print('adding ' + str(val) + ' at pos:' + str(((pos+1)//9,(pos+1)%9)))
            if self._solver():
                return True
        self.board[pos//9][pos%9] = 0
        return False

In [3]:
bd = Board()

##Not legal board, but ok on rows
# for i in range(9):
#     options[i] = options[i] + i + 1
#     for j in range(9):
#         board[j][i] = board[j][i] + i + 1

##Not legal board, but ok on boxes
# for i in range(3):
#     for j in range(3):
#         board[3*i][3*j:3*j+3] = [1,2,3]
#         board[3*i+1][3*j:3*j+3] = [4,5,6]
#         board[3*i+2][3*j:3*j+3] = [7,8,9]

##Legal Board
# for i in range(3):
#     for j in range(9):
#         board[i+0][j] = (3*i+j+0)%9 + 1
#         board[i+3][j] = (3*i+j+1)%9 + 1
#         board[i+6][j] = (3*i+j+2)%9 + 1

###Here, you can create a board, set values, and view the board.

In [4]:
bd = Board()
bd.set_val(3,35)
bd.set_val(5,(7,3))
bd.set_row('12345678.',1)
# bd.set_column([9,8,7,6,5,4,3,2,1],9)
# bd.set_board([i%9 for i in range(81)])
print(bd.check_legal())
bd.view()

True


array([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  5.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

###Solving is done with a brute-force, recursive algorithm (sequentially trying each value in each unfilled square). 
Regardless, it is reasonably fast for many boards :-)

In [5]:
%time bd.solve()

Solve complete!
Wall time: 522 ms


array([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
       [ 4.,  5.,  6.,  7.,  8.,  9.,  1.,  2.,  3.],
       [ 7.,  8.,  9.,  1.,  2.,  3.,  5.,  4.,  6.],
       [ 2.,  1.,  4.,  5.,  6.,  7.,  9.,  3.,  8.],
       [ 3.,  6.,  5.,  2.,  9.,  8.,  4.,  1.,  7.],
       [ 8.,  9.,  7.,  3.,  1.,  4.,  2.,  6.,  5.],
       [ 5.,  3.,  1.,  6.,  7.,  2.,  8.,  9.,  4.],
       [ 6.,  7.,  8.,  9.,  4.,  1.,  3.,  5.,  2.],
       [ 9.,  4.,  2.,  8.,  3.,  5.,  6.,  7.,  1.]])

###Enjoy! Use this notebook to play with Sudoku boards or solve puzzles that you're stuck on. (Or, check if your guesses are on the right track!)
For example, here is a [list](http://www.puzzles.ca/sudoku.html) of Sudoku puzzles.

In [6]:
##This is Easy Puzzle #217 from http://www.puzzles.ca/sudoku.html
##Any nonzero character is interpreted as "0" (unknown)
vals =  '000090010300010200000400067' +\
        '007500000006029081509000000' +\
        '200000000008000000160305002'
print(len(vals), type(vals))

(81, <type 'str'>)


In [7]:
bd = Board()
bd.set_board(vals)
bd.view()

array([[ 0.,  0.,  0.,  0.,  9.,  0.,  0.,  1.,  0.],
       [ 3.,  0.,  0.,  0.,  1.,  0.,  2.,  0.,  0.],
       [ 0.,  0.,  0.,  4.,  0.,  0.,  0.,  6.,  7.],
       [ 0.,  0.,  7.,  5.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  6.,  0.,  2.,  9.,  0.,  8.,  1.],
       [ 5.,  0.,  9.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  8.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  6.,  0.,  3.,  0.,  5.,  0.,  0.,  2.]])

In [8]:
%time bd.solve()

Solve complete!
Wall time: 12.9 s


array([[ 6.,  7.,  2.,  8.,  9.,  3.,  4.,  1.,  5.],
       [ 3.,  4.,  5.,  6.,  1.,  7.,  2.,  9.,  8.],
       [ 9.,  8.,  1.,  4.,  5.,  2.,  3.,  6.,  7.],
       [ 8.,  1.,  7.,  5.,  3.,  4.,  6.,  2.,  9.],
       [ 4.,  3.,  6.,  7.,  2.,  9.,  5.,  8.,  1.],
       [ 5.,  2.,  9.,  1.,  6.,  8.,  7.,  3.,  4.],
       [ 2.,  5.,  3.,  9.,  7.,  1.,  8.,  4.,  6.],
       [ 7.,  9.,  8.,  2.,  4.,  6.,  1.,  5.,  3.],
       [ 1.,  6.,  4.,  3.,  8.,  5.,  9.,  7.,  2.]])

###Of course, there are much faster implementations of this code. Also, a very small number of boards (~0.05%) will take a long, long time to solve. See Peter Norvig's [writeup](http://norvig.com/sudoku.html) for more information.


In [9]:
hard1  =    '.....6....59.....82....8...' +\
            '.45........3........6..3.54' +\
            '...325..6..................'
bd.set_board(hard1)
bd.view()
##This takes a very long time!
#bd.solve()

array([[ 0.,  0.,  0.,  0.,  0.,  6.,  0.,  0.,  0.],
       [ 0.,  5.,  9.,  0.,  0.,  0.,  0.,  0.,  8.],
       [ 2.,  0.,  0.,  0.,  0.,  8.,  0.,  0.,  0.],
       [ 0.,  4.,  5.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  6.,  0.,  0.,  3.,  0.,  5.,  4.],
       [ 0.,  0.,  0.,  3.,  2.,  5.,  0.,  0.,  6.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

###Have fun!

In [10]:
bd = Board()
bd.view()

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])